# Import Libraries

In [2]:
import numpy as np
from astropy.table import Table
import pandas as pd

In [3]:
observatory = '/data/mfonseca/'
my_computer = '/home/polaris/Lab_Astro/data/'

directory = my_computer

In [18]:
matched_no_mags_path = directory + 'cross_match/EMU_0102-32_DESY6GOLD_VIKING_CATWISE_noMags/EMU_0102-32_DESY6GOLD_VIKINGDR5_CATWISE_noMag.fits'
matched_no_mags = Table.read(matched_no_mags_path)
matched_no_mags = matched_no_mags.to_pandas()
match_mask = (matched_no_mags['match_flag'] == 1) & (matched_no_mags['p_i'] >= 0.1) & (matched_no_mags['p_any'] >=0.83)
catalog_matches_noMags = matched_no_mags[match_mask]
full_match_mask = (catalog_matches_noMags['DESY6_coadd_object_id'] != -99) & (catalog_matches_noMags['VKG_sourceID'] != -99) & (catalog_matches_noMags['CAT_source_id'] != b'-99')
catalog_matches_noMags_full = catalog_matches_noMags[full_match_mask]

print(catalog_matches_noMags_full.columns.tolist())

['EMU_island_id', 'EMU_island_name', 'EMU_n_components', 'EMU_ra_deg_cont', 'EMU_dec_deg_cont', 'EMU_freq', 'EMU_maj_axis', 'EMU_min_axis', 'EMU_pos_ang', 'EMU_flux_int', 'EMU_flux_int_err', 'EMU_flux_peak', 'EMU_mean_background', 'EMU_background_noise', 'EMU_max_residual', 'EMU_min_residual', 'EMU_mean_residual', 'EMU_rms_residual', 'EMU_stdev_residual', 'EMU_x_min', 'EMU_x_max', 'EMU_y_min', 'EMU_y_max', 'EMU_n_pix', 'EMU_solid_angle', 'EMU_beam_area', 'EMU_x_ave', 'EMU_y_ave', 'EMU_x_cen', 'EMU_y_cen', 'EMU_x_peak', 'EMU_y_peak', 'EMU_flag_i1', 'EMU_flag_i2', 'EMU_flag_i3', 'EMU_flag_i4', 'EMU_comment', 'DESY6_coadd_object_id', 'DESY6_ra', 'DESY6_dec', 'DESY6_mag_auto_g', 'DESY6_magerr_auto_g', 'DESY6_mag_auto_r', 'DESY6_magerr_auto_r', 'DESY6_mag_auto_i', 'DESY6_magerr_auto_i', 'DESY6_mag_auto_z', 'DESY6_magerr_auto_z', 'DESY6_mag_auto_y', 'DESY6_magerr_auto_y', 'DESY6_a_fiducial_g', 'DESY6_a_fiducial_i', 'DESY6_a_fiducial_r', 'DESY6_a_fiducial_y', 'DESY6_a_fiducial_z', 'DESY6_spre

# Define column names

In [5]:
id = ['EMU_island_name']

redshift = ['DESY6_dnf_z']

des_filters_cigale = ['ctio_decam.g', 'ctio_decam.r', 'ctio_decam.i', 'ctio_decam.z', 'ctio_decam.Y']
des_errors_cigale = ['ctio_decam.g_err', 'ctio_decam.r_err', 'ctio_decam.i_err', 'ctio_decam.z_err', 'ctio_decam.Y_err']

viking_filters_cigale = ['paranal.vircam.Z', 'paranal.vircam.Y', 'paranal.vircam.J', 'paranal.vircam.H', 'paranal.vircam.Ks']
viking_errors_cigale = ['paranal.vircam.Z_err', 'paranal.vircam.Y_err', 'paranal.vircam.J_err', 'paranal.vircam.H_err', 'paranal.vircam.Ks_err']

catwise_filters_cigale = ['wise.W1', 'wise.W2']
catwise_errors_cigale = ['wise.W1_err', 'wise.W2_err']


In [6]:
extra_cols_dict = {
    'EMU_island_name' : 'id',
    'DESY6_dnf_z' : 'redshift',
}

des_cols_dict = {
    'DESY6_mag_auto_g_flux': 'ctio.decam.g',
    'DESY6_mag_auto_r_flux': 'ctio.decam.r',
    'DESY6_mag_auto_i_flux': 'ctio.decam.i',
    'DESY6_mag_auto_z_flux': 'ctio.decam.z',
    'DESY6_mag_auto_y_flux': 'ctio.decam.Y',
    'DESY6_magerr_auto_g_fluxerr' : 'ctio.decam.g_err',
    'DESY6_magerr_auto_r_fluxerr' : 'ctio.decam.r_err',
    'DESY6_magerr_auto_i_fluxerr' : 'ctio.decam.i_err',
    'DESY6_magerr_auto_z_fluxerr' : 'ctio.decam.z_err',
    'DESY6_magerr_auto_y_fluxerr' : 'ctio.decam.Y_err',
}

viking_cols_dict = {
    'VKG_zAperMag3_flux': 'paranal.vircam.Z',
    'VKG_yAperMag3_flux': 'paranal.vircam.Y',
    'VKG_jAperMag3_flux': 'paranal.vircam.J',
    'VKG_hAperMag3_flux': 'paranal.vircam.H',
    'VKG_ksAperMag3_flux': 'paranal.vircam.Ks',
    'VKG_zAperMag3Err_fluxerr': 'paranal.vircam.Z_err',
    'VKG_yAperMag3Err_fluxerr': 'paranal.vircam.Y_err',
    'VKG_jAperMag3Err_fluxerr': 'paranal.vircam.J_err',
    'VKG_hAperMag3Err_fluxerr': 'paranal.vircam.H_err',
    'VKG_ksAperMag3Err_fluxerr': 'paranal.vircam.Ks_err',

}

catwise_cols_dict = {
    'CAT_w1mpro_flux': 'wise.W1',
    'CAT_w2mpro_flux': 'wise.W2',
    'CAT_w1sigmpro_fluxerr': 'wise.W1_err',
    'CAT_w2sigmpro_fluxerr': 'wise.W2_err',
}

all_dicts = [extra_cols_dict, des_cols_dict, viking_cols_dict, catwise_cols_dict]
mag_cols = [
    'DESY6_mag_auto_g',
    'DESY6_mag_auto_r',
    'DESY6_mag_auto_i',
    'DESY6_mag_auto_z',
    'DESY6_mag_auto_y',
    'VKG_zAperMag3',
    'VKG_yAperMag3',
    'VKG_jAperMag3',
    'VKG_hAperMag3',
    'VKG_ksAperMag3',
    'CAT_w1mpro',
    'CAT_w2mpro'
]

mag_err_cols = [
    'DESY6_magerr_auto_g',
    'DESY6_magerr_auto_r',
    'DESY6_magerr_auto_i',
    'DESY6_magerr_auto_z',
    'DESY6_magerr_auto_y',
    'VKG_zAperMag3Err',
    'VKG_yAperMag3Err',
    'VKG_jAperMag3Err',
    'VKG_hAperMag3Err',
    'VKG_ksAperMag3Err',
    'CAT_w1sigmpro',
    'CAT_w2sigmpro'
]

# Add custom DES filters

### Normalize the transmission

In [ ]:
g_filter_path = '/data/mfonseca/survey_data/DES_data/CTIO_DECam.g.dat'
i_filter_path = '/data/mfonseca/survey_data/DES_data/CTIO_DECam.i.dat'
r_filter_path = '/data/mfonseca/survey_data/DES_data/CTIO_DECam.r.dat'
z_filter_path = '/data/mfonseca/survey_data/DES_data/CTIO_DECam.z.dat'
y_filter_path = '/data/mfonseca/survey_data/DES_data/CTIO_DECam.Y.dat'

In [ ]:
def normalize_filter_data(input_filename, output_filename):
    """
    Normalizes the second column (transmission values) of a .dat file.

    Parameters:
    - input_filename (str): Path to the input .dat file.
    - output_filename (str): Path where the normalized data will be saved.

    The input file should contain two columns: wavelength (first column) and transmission (second column).
    """
    # Initialize lists to store data
    lambda_values = []
    transmission_values = []

    # Read the data from the .dat file
    with open(input_filename, 'r') as f:
        for line in f:
            # Skip comments or empty lines
            if line.startswith('#') or line.strip() == '':
                continue
            
            # Split the line into wavelength and transmission
            columns = line.split()
            lambda_values.append(float(columns[0]))  # Wavelength (first column)
            transmission_values.append(float(columns[1]))  # Transmission (second column)

    # Convert the lists to numpy arrays for easier processing
    lambda_values = np.array(lambda_values)
    transmission_values = np.array(transmission_values)

    # Normalize the transmission values (second column)
    max_transmission = np.max(transmission_values)
    normalized_transmission = transmission_values / max_transmission

    # Save the normalized data to the output .dat file
    with open(output_filename, 'w') as f:
        # Optionally add a header
        f.write("# Wavelength (Å) and Normalized Transmission Values\n")
        
        # Write the wavelength and the normalized transmission
        for lambda_val, norm_trans in zip(lambda_values, normalized_transmission):
            f.write(f"{lambda_val} {norm_trans}\n")

    print(f"Normalization complete. Output saved to {output_filename}.")

In [ ]:
normalize_filter_data(g_filter_path, g_filter_path)
normalize_filter_data(i_filter_path, i_filter_path)
normalize_filter_data(r_filter_path, r_filter_path)
normalize_filter_data(y_filter_path, y_filter_path)
normalize_filter_data(z_filter_path, z_filter_path)


# Convert magnitudes to fluxes

In [30]:
def mag_to_flux_mJy(mag):
    '''
    Convert magnitude to flux using the formula:
    flux_mJy = 3631 * 10 ** (-0.4 * mag)

    Args:
        mag (float or array-like): Magnitude value(s) to convert.

    Returns:
        float or array-like: Corresponding flux value(s) in mJy.

    '''
    return 3631 * 10 ** (-0.4 * mag)

def magerr_to_fluxerr_mJy(mag, magerr):
    '''
    Convert magnitude error to flux error using the formula:
    fluxerr_mJy = 3631 * ln(10) * 10 ** (-0.4 * mag) * (-0.4) * magerr

    Args:
        mag (float or array-like): Magnitude value(s) to convert.
        magerr (float or array-like): Magnitude error value(s) to convert.

    Returns:
        float or array-like: Corresponding flux error value(s) in mJy.
    
    '''

    return 3631 * np.log(10) * 10 ** (-0.4 * mag) * (0.4) * magerr

In [31]:
def add_flux_to_df(df, mag_cols, mag_err_cols):
    '''
    Add flux and flux error columns to the DataFrame.

    Args:
        df (DataFrame): Input DataFrame.
        mag_cols (list): List of magnitude column names.
        mag_err_cols (list): List of magnitude error column names.

    Returns:
        DataFrame: Updated DataFrame with flux and flux error columns added.
    
    '''

    df = df.copy()

    for mag_col, mag_err_col in zip(mag_cols, mag_err_cols):
        # Create new column names for flux and flux error
        flux_col = f"{mag_col}_flux"
        flux_err_col = f"{mag_err_col}_fluxerr"

        # Convert magnitudes to fluxes
        df[flux_col] = mag_to_flux_mJy(df[mag_col])

        # Convert magnitude errors to flux errors
        df[flux_err_col] = magerr_to_fluxerr_mJy(df[mag_col], df[mag_err_col])

    return df

# Create a catalog for CIGALE

Creates an ASCII file for cigale 

In [32]:
def generate_ascii_with_flux(catalog_df, mag_columns, mag_error_columns, dict_all_cols, n_rows ,output_path):
    """
    Converts magnitude columns and magnitude error columns to flux and flux error in mJy.

    Args:
        df (pd.DataFrame): DataFrame with magnitude columns and their corresponding error columns.
        mag_columns (list): List of magnitude column names to convert.
        mag_error_columns (list): List of magnitude error column names to convert.
        dict_flux_to_filters (dict): Dictionary mapping flux columns to their corresponding filters.

    Returns:
        pd.DataFrame: DataFrame with new flux and flux error columns added.
    """

    catalog_flux = add_flux_to_df(catalog_df, mag_columns, mag_error_columns)

    print(catalog_flux.columns.tolist())

    # Create a new DataFrame to store the data for CIGALE
    cigale_df = pd.DataFrame()

    # Add the ID, redshift columns, flux and flux_err columns
    for catalog in dict_all_cols:
        for col, new_col in catalog.items():
            cigale_df[new_col] = catalog_flux[col]

    cigale_to_save = cigale_df.head(n_rows)

    cigale_to_save.to_csv(output_path, sep=' ', index=False)

    print(f'Saved Cigale file to path {output_path}')
    

In [37]:
generate_ascii_with_flux(catalog_matches_noMags_full, mag_cols, mag_err_cols, all_dicts, 5, directory + 'cigale/test4/test4' )

['EMU_island_id', 'EMU_island_name', 'EMU_n_components', 'EMU_ra_deg_cont', 'EMU_dec_deg_cont', 'EMU_freq', 'EMU_maj_axis', 'EMU_min_axis', 'EMU_pos_ang', 'EMU_flux_int', 'EMU_flux_int_err', 'EMU_flux_peak', 'EMU_mean_background', 'EMU_background_noise', 'EMU_max_residual', 'EMU_min_residual', 'EMU_mean_residual', 'EMU_rms_residual', 'EMU_stdev_residual', 'EMU_x_min', 'EMU_x_max', 'EMU_y_min', 'EMU_y_max', 'EMU_n_pix', 'EMU_solid_angle', 'EMU_beam_area', 'EMU_x_ave', 'EMU_y_ave', 'EMU_x_cen', 'EMU_y_cen', 'EMU_x_peak', 'EMU_y_peak', 'EMU_flag_i1', 'EMU_flag_i2', 'EMU_flag_i3', 'EMU_flag_i4', 'EMU_comment', 'DESY6_coadd_object_id', 'DESY6_ra', 'DESY6_dec', 'DESY6_mag_auto_g', 'DESY6_magerr_auto_g', 'DESY6_mag_auto_r', 'DESY6_magerr_auto_r', 'DESY6_mag_auto_i', 'DESY6_magerr_auto_i', 'DESY6_mag_auto_z', 'DESY6_magerr_auto_z', 'DESY6_mag_auto_y', 'DESY6_magerr_auto_y', 'DESY6_a_fiducial_g', 'DESY6_a_fiducial_i', 'DESY6_a_fiducial_r', 'DESY6_a_fiducial_y', 'DESY6_a_fiducial_z', 'DESY6_spre